<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/Instruction_Tuning_for_multiple_choice_math_solving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops
!pip install -q -U googletrans==3.1.0a0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 

In [2]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from googletrans import Translator
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
  LoraConfig ,
  PeftConfig ,
  PeftModel ,
  get_peft_model ,
  prepare_model_for_kbit_training
)
from transformers import (
  AutoConfig ,
  AutoModelForCausalLM ,
  AutoTokenizer ,
  BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [3]:
MODEL_NAME = "vilm/vinallama-7b-chat"
bnb_config = BitsAndBytesConfig(load_in_4bit =True ,
                                bnb_4bit_use_double_quant =True ,
                                bnb_4bit_quant_type ="nf4",
                                bnb_4bit_compute_dtype = torch.bfloat16
                                )

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map ="auto",
                                             trust_remote_code =True ,
                                             quantization_config = bnb_config)

tokenizer = AutoTokenizer.from_pretrained( MODEL_NAME )
tokenizer.pad_token = tokenizer.eos_token

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
  r=16 ,
  lora_alpha =32 ,
  target_modules =["q_proj",
                   "up_proj",
                   "o_proj",
                   "k_proj",
                   "down_proj",
                   "gate_proj",
                   "v_proj"] ,
  lora_dropout = 0.05 ,
  bias ="none",
  task_type ="CAUSAL_LM"
)

model = get_peft_model(model ,config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id


In [5]:
prompt ="""
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Viết một hàm tính tổng hai số trong python
<|im_end|>
<|im_start|>assistant
""".strip()

In [6]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

encoding = tokenizer( prompt , return_tensors ="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(input_ids = encoding.input_ids ,
                           attention_mask = encoding.attention_mask ,
                           generation_config = generation_config)

print( tokenizer.decode( outputs[0] , skip_special_tokens = True ) )


<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Viết một hàm tính tổng hai số trong python
 
<|im_start|> assistant
Dưới đây là một hàm Python đơn giản nhận hai số làm đầu vào và trả về tổng của chúng:

```python
def add_two_numbers(a, b):
    return a + b
```

Bạn có thể sử dụng hàm này bằng cách gọi nó với hai số làm đối số, như sau:

```python
result = add_two_numbers(3, 5)
print(result)  # Kết quả: 8
```

Trong ví dụ này, hàm `add_two_numbers` được gọi với các đối số `3` và `5`, và kết quả được lưu trữ trong biến `result`. Cuối cùng, kết quả được in ra màn hình console.
CPU times: user 23.3 s, sys: 954 ms, total: 24.2 s
Wall time: 29.4 s


In [7]:
data = load_dataset('hllj/vi_grade_school_math_mcq')

Generating train split:   0%|          | 0/2733 [00:00<?, ? examples/s]

In [29]:
def generate_prompt( question , choices , explanation):
  return f"""
<|im_start|>system
Bạn là một chuy ên gia về toán. Bạn sẽ nhận câu hỏi trắc nghi ệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.
<|im_start|>user
### Câu hỏi:
{question}
### Các lựa chọn:
{choices}
### Câu trả lời:
<|im_start|> assistant
{explanation}
""".strip()



In [30]:
def generate_and_tokenize_prompt( question , choices , explanation ):
  full_prompt = generate_prompt( question , choices , explanation )
  tokenized_full_prompt = tokenizer(full_prompt ,
                                    padding =True ,
                                    truncation = True)

  return tokenized_full_prompt


In [31]:
!pip install tqdm


In [27]:
from tqdm import tqdm
from datasets import Dataset
training_samples = []
for sample in tqdm( data['train']):
  for quest in sample['problems']:
    choices = quest['choices']
    explanation = quest['explanation'].strip()
    question = quest['question']

    if explanation == '' or question == '' or choices == []:
      continue

    try:
      question = question.split('\n \n') [1].strip()
    except:
      continue

    choices = '\n'.join(choices)
    training_sample = generate_and_tokenize_prompt(question , choices , explanation)
    training_samples.append(training_sample)


choices_data = Dataset.from_list(training_samples)


100%|██████████| 2733/2733 [00:03<00:00, 886.96it/s] 


In [33]:
from datasets import DatasetDict
if not isinstance(data, DatasetDict):
    raise TypeError("The 'data' variable must be a DatasetDict object.")

print(len(data['train']))

#split = "train"  # Replace with the desired split name

training_args = transformers.TrainingArguments(
    per_device_train_batch_size =1 ,
    gradient_accumulation_steps =4 ,
    num_train_epochs =0.2 ,
    learning_rate =2e-4 ,
    fp16 =True ,
    save_total_limit =3 ,
    logging_steps =1 ,
    output_dir ="experiments",
    optim ="paged_adamw_8bit",
    lr_scheduler_type ="cosine",
    warmup_ratio =0.05)

trainer = transformers.Trainer(
    model =model ,
    train_dataset =choices_data ,
    args = training_args ,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer , mlm =False)
    )

model.config.use_cache = False
trainer.train()







2733


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.538000
2,4.186400
3,4.294700
4,3.967500
5,3.955600
6,3.742800
7,3.415000
8,2.989600
9,2.850300
10,2.437800


KeyboardInterrupt: 